# Navigation

## Preparation

In [1]:
from unityagents import UnityEnvironment
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from collections import deque
from dqn_agent import Agent

In [2]:
def dqn(brain_name, agent, n_episodes=700, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
      
    scores = []                        
    scores_window = deque(maxlen=100)
    scores_pass = 13
    eps = eps_start 
    solved = False
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]   
        score = 0
        while True:
            action = agent.act(state, eps)            
            brain_info = env.step(action)[brain_name]
            next_state = brain_info.vector_observations[0]
            reward = brain_info.rewards[0]
            done = brain_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)
        scores.append(score)
        eps = max(eps_end, eps_decay*eps) 
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=scores_pass:
            if solved == False:
                print('\n * Environment first solved in {:d} episodes! Average Score: {:.2f}. Continue training...'.format(i_episode, np.mean(scores_window)))
                solved = True
            scores_pass = np.mean(scores_window)
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    return scores

def plot_score(scores, rolling_window=100):
    df_scores = pd.DataFrame(scores, columns=['EpisodeScore'])
    df_scores['MovingAvg100'] = df_scores['EpisodeScore'].rolling(rolling_window).mean()
    df_scores.plot(color=['grey', 'red'])
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

## Environment

In [3]:
# Environment
env = UnityEnvironment(file_name="Banana\Banana.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


## Train an Agent

In [4]:
# Train or load model
train_model = True

In [5]:
# Agent
if train_model:
    action_size = brain.vector_action_space_size
    state_size = brain.vector_observation_space_size
    agent = Agent(state_size=state_size, action_size=action_size, device=device)
    scores = dqn(brain_name, agent)
    plot_score(scores)
else:
    agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

Episode 100	Average Score: 0.96
Episode 200	Average Score: 4.39
Episode 228	Average Score: 5.61

KeyboardInterrupt: 

## Watch the Smart Agent

In [ ]:
# Watch the trained agent
env_info = env.reset(train_mode=False)[brain_name] 
state = env_info.vector_observations[0]
score = 0
while True:
    action = agent.act(state, eps=0)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0] 
    done = env_info.local_done[0]
    score += reward
    state = next_state
    if done: 
        break
print("Score: {}".format(score))
# env.close()